In [1]:
import pandas as pd
import glob
import os
import gutenberg
from gutenberg.cleanup import strip_headers
import sqlite3 as sql
from ast import literal_eval

In [2]:
conn = sql.connect('pg-pr-eng.db')
c = conn.cursor()

In [6]:
df = pd.read_sql('show tables', conn)

DatabaseError: Execution failed on sql 'show tables': near "show": syntax error

In [84]:
def getText(filename): 
    if filename is not None and filename != "None": 
        if os.path.isfile(filename): 
            print('Loading %s' % filename, end='\r')
            try: 
                text = open(filename).read()
            except: 
                text = open(filename, encoding='latin-1').read()
            return text
        else: 
            print("Couldn't find %s" % filename, end='\r')
            return None
    else: 
        return None

In [89]:
for i, row in df.iterrows(): 
    if row.filename is not None and row.filename != "None":
        # Use Gitenberg file if available. 
        filename = row.filename
    elif row.gutFilename is not None and row.gutFilename != "None":
        filename = row.gutFilename
    bookID = row.id
    text = getText(filename)
    print(row['index'], end='\r')
    c.execute('update pg set "text" = ? where "id" == ?', [text, bookID])

In [ ]:
for i, row in df.iterrows(): 
    if row.text is not None and row.text != "None":
        text = strip_headers(row.text)
    bookID = row.id
    print(row['index'], end='\r')
    c.execute('update pg set "text_cleaned" = ? where "id" == ?', [text, bookID])

In [9]:
conn.commit()

In [ ]:
i

In [ ]:
df.to_sql('pg', conn, if_exists='replace')

/usr/lib/python3.6/site-packages/pandas/core/generic.py:1201: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)
